In [15]:
from datasets import load_dataset
from model import NERClassifier
from preprocess_dataset import NERDataset 
from trainner import Trainner
from transformers import BertTokenizer

import torch

In [16]:

CHECKPOINT = 'neuralmind/bert-base-portuguese-cased'
tokenizer = BertTokenizer.from_pretrained(CHECKPOINT)
data = "lener_br"
dataset = load_dataset(data)

Reusing dataset lener_br (/home/caiotulio/.cache/huggingface/datasets/lener_br/lener_br/1.0.0/4a8c97e6813b5c2d85a50faf0a3e6c24ea82f4a9044e6e9e8b24997d27399382)


In [17]:
pytorch_dataset_train = NERDataset(data=dataset['train'], max_len=256, tokenizer=tokenizer)
teste = pytorch_dataset_train[0]
n_labels = 12 #TODO: por enquanto hardcodado 
model = NERClassifier(n_labels=n_labels, checkpoint=CHECKPOINT)
input_ids = teste['input_ids']
attention_mask = teste['attention_mask']

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model check

In [18]:
print(input_ids)
print(input_ids.shape)
print('---------------------')
print(attention_mask)
print(attention_mask.shape)
print('---------------------')

tensor([  192,  7463,  8427, 22301,   131, 12127,  9008, 22301, 22402, 16484,
          187, 22360, 22339,  9008,   118,   177, 22402, 16484, 10836, 13760,
         7545, 22320, 22323, 22351, 22301, 22402, 16484,   212,  8718,   250,
         7665,  6072,   213,  8718, 22301,  6538,   118, 11635,  9008, 13270,
         7073,  6765,   118, 11741, 22328, 22341,  6392, 22301,   212,  9008,
        22317,   213,  7073,  6538, 22321, 22352, 21748, 22317,   212, 22371,
        22318, 22327,  6162, 22317,   192, 22311,   278,  5650, 22341,   257,
         5476, 15289,  5903, 22327,   118,   248, 18199,  6392, 11836, 22309,
          118,   177, 10409, 22420, 22320, 14298, 22301, 10836, 13760, 16017,
        22322, 22339, 12547, 22402, 16484, 15040, 18868, 22322, 22349, 22341,
         9208,   248, 22301, 13760, 11846, 22379, 22320, 14298, 22301,   177,
         5226, 22341, 22317,   118, 11635,  3341, 12547, 22402, 22301, 10836,
        13760,  4529,  5869, 22351,   118, 11635, 22309, 22333, 

In [13]:
a = input_ids.tolist()
b = attention_mask.tolist()

In [19]:
saida = model(input_ids, attention_mask)
print(saida)

ValueError: not enough values to unpack (expected 2, got 1)